In [1]:
%load_ext autoreload
%autoreload 2
import os
# import sys
# sys.path.append( '/anaconda/envs/py37_pytorch/lib/python3.7')
# sys.path.append('/anaconda/envs/py37_pytorch/lib/python3.7/site-packages')
# sys.path.append('/home/v-zeyyan/.local/lib/python3.6/site-packages')
import time
import yaml
import glob
import logging
import numpy as np
from importlib import reload  # Not needed in Python 2

import torch
from torch import nn
import torch.nn.functional as F
from torchvision.models import ResNet
from torch.utils.data import DataLoader,Dataset
import seaborn as sns


from dataset.ASdataset import AS_Data
from dataset.ASdataset_obs_train_input import AS_Data_obs

device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")
reload(logging)
logging.basicConfig(level=logging.INFO,#控制台打印的日志级别
                    filename='logging.txt',
                    filemode='a',##模式，有w和a，w就是写模式，每次都会重新写日志，覆盖之前的日志
                    #a是追加模式，默认如果不写的话，就是追加模式
                    format=
                    '%(asctime)s : %(message)s',
                    )
def print_log(string):
    logging.info(string)
    print(string)
    
    
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

voc_idx = np.array([0,2,3,4,8,9,10] + list(range(32,51)))


In [2]:
def show_station_diff(month_idx = 0):

    for month in [month_idx]:
        for p in range(len(Data.pollution_idx)):
            p_name = pollution[p]

            month_obs = Data.obs_label[month][24:,p].copy()
            month_nn = Data.finetune_label[month][24:,p].copy()
            month_ctm = Data.label[month][24:,p].copy()

#             print(month_ctm.shape)
#             print(month_nn.shape)
#             print(month_obs.shape)
            month_ctm = month_ctm[528:]
            month_nn = month_nn[528:]
            month_obs = month_obs[528:]
#             print(month_ctm.shape)
#             print(month_nn.shape)
#             print(month_obs.shape)

            count = np.sum(month_obs==-999)
            
            # pixel with no obersertion -> 0
            month_ctm[month_obs==-999] = 0
            month_nn[month_obs==-999] = 0
            month_obs[month_obs==-999] = 0
            
            MAE = np.sum(np.abs(month_ctm-month_nn))/(len(month_ctm)*182*232-count)
            print('MAE: ',MAE)
            
            sum_month_obs = np.sum(month_obs,axis = 0)
            sum_month_nn = np.sum(month_nn,axis = 0)
            sum_month_ctm = np.sum(month_ctm,axis = 0)
            
#             print(sum_month_obs.shape)
#             print(sum_month_nn.shape)
#             print(sum_month_ctm.shape)
            
            plt.rcParams['figure.figsize'] = (10,5)

            a = ((sum_month_nn - sum_month_ctm)/(sum_month_ctm+1))
#             print(sum_month_obs)
            print(a.shape)
            print(a)
#             a = a.reshape((1,42224))
            print(np.mean(np.abs(a)))
#             image_show(a.reshape((1,182,232)))
            a = a[sum_month_obs>0]
            plt.hist(a,bins = 100,range = (-1,1))
            plt.title(f'{p_name}: (nn-ctm)/(1+ctm):{np.mean(np.abs(a))}')
            plt.show()
            

            a = ((sum_month_nn - sum_month_ctm)/(sum_month_obs+1))
#             image_show(a.reshape((1,182,232)))
            a = a[sum_month_obs>0]
            plt.hist(a,bins = 100,range = (-1,1))
            plt.title(f'{p_name}: (nn-ctm)/(1+obs):{np.mean(np.abs(a))}')
            plt.show()

            a = ((sum_month_nn - sum_month_obs)/(sum_month_obs+1))
#             image_show(a.reshape((1,182,232)))
            a = a[sum_month_obs>0]
            plt.hist(a,bins = 100,range = (-1,1))
            plt.title(f'{p_name}: (nn-obs)/(1+obs):{np.mean(np.abs(a))}')
            plt.show()

            a = ((sum_month_ctm - sum_month_obs)/(sum_month_obs+1))
#             image_show(a.reshape((1,182,232)))
            a = a[sum_month_obs>0]
            plt.hist(a,bins = 100,range = (-1,1))
            plt.title(f'{p_name}: (ctm-obs)/(1+obs):{np.mean(np.abs(a))}')
            plt.show()


            plt.rcParams['figure.figsize'] = (15, 5)

#             for b,a,name in [[136,115,'beijing'],[139,112,'tianjin'],[130,107,'shijiazhuang'],[157,76,'ningbo'],[96,99,'lanzhou'],[112,91,'xian'],[48,139,'xinjiang']]:
            for b,a,name in [[114,26,'sanya'],[140,82,'hefei'],[155,81,'shanghai'],[128,46,'guangzhou'],[146,53,'xiamen']]:
                obs = month_obs[:,a,b]
                nn = month_nn[:,a,b]
                ctm = month_ctm[:,a,b]

#                 plt.plot(obs[obs!=-999],'k')
                plt.plot(nn[obs!=-999],'r--')
                plt.plot(ctm[obs!=-999],'g--')

#                 plt.legend(['obs','nn_label','ctm_label'])
                plt.legend(['nn_label','ctm_label'])
                plt.title(p_name + ' : ' + name)
                plt.show()



In [3]:
with open('config/cfg.yaml','r') as f:
    cfg = yaml.load(f)
    
cfg = {**cfg['step2'],**cfg['share_cfg'],**cfg['share_cfg']['data_path']}
T = cfg['T']
pollution = cfg['pollution']
batch_size = cfg['batch_size']

name = cfg['name']
### remove CH4,AACD,ACET
EM_idx = np.array(cfg['EM_idx'])
EM_save_path = cfg['EM_save_path']

print_log('training data is loading')
Data = AS_Data_obs(cfg,left = 0,right = 1,window = T,EM_idx = EM_idx,pollution = pollution)
trainloader = DataLoader(Data,batch_size=batch_size,shuffle=True)
print_log('training data load successful')

print_log(len(Data))
print_log(Data.EM_idx)

/anaconda/envs/py37_default/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


training data is loading
['PM25', 'O3', 'NO2', 'SO2']
/AS_data/Conc_npy/TOTAL_2015_01_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy   is loading
(744, 4, 182, 232)
/AS_data/zeyuan_folder/concat_data/rest_EM_2015_01_744_07_182_232.npy   is loading
(744, 7, 182, 232)
/AS_data/METCRO2D_npy/METCRO2D_2015_01_744_34_182_232.npy   is loading
[0, 721]
/AS_data/obs_npy/obs2015_01_744_6_182_232.npy   is loading
/AS_data/obs_npy/obs2015_04_720_6_182_232.npy   is loading
/AS_data/obs_npy/obs2015_07_744_6_182_232.npy   is loading
/AS_data/obs_npy/obs2015_10_744_6_182_232.npy   is loading
training data load successful
720
[1 2 4 5 6]


In [6]:
from model.res_model_LSTM import res8
from model.unet_model_LSTM import UNet
from model.layers import Tensor_Parameter
from util.optim import SGD,Adam
# optimizer = torch.optim.SGD(t2p.parameters(),lr=1)
china_sign = (Data.obs_label[0][24,0]!=-999)

# name = 'res_2layer_correctdata'
# test_model.load_state_dict(torch.load('model_save/res_2layer_9_epoch.t'))
test_model = UNet(cfg['meteorological_dim']+cfg['emission_dim'],cfg['grid_dim'],T=T,bilinear=False,pre_dim = len(pollution)) #+80
# test_model = res8(cfg['meteorological_dim']+cfg['emission_dim'],cfg['grid_dim'],T=T,pre_dim = len(pollution))
t2p = Tensor_Parameter()


test_model.to(device)
t2p.to(device)
criterion = torch.nn.L1Loss()

optimizer = Adam(t2p.parameters(),china_sign,trust = 1,lr=1e-2)
# test_model.load_state_dict(torch.load(name))
test_model.load_state_dict(torch.load('model_save/step1_4month_epoch8.t'))


<All keys matched successfully>

In [7]:
def score(model,loader,criterion= nn.L1Loss() ):
    model.eval()
    ls = []
    for idx,i in enumerate(loader):
        with torch.no_grad():
            indexes,input,grid,yt_1,label,obs = i[0],i[1],i[2],i[3],i[4],i[5]
            input,grid,yt_1,label,obs = input.to(device),grid.to(device),yt_1.to(device),label.to(device),obs.float().to(device)
            input = t2p(input)
            y_pred = model(input,grid,yt_1)
            
#             Data.update_labels(indexes,y_pred)
            if torch.sum(obs!=-999)==0:
                continue
            
            cur_loss = []
            for pollution in range(y_pred.shape[1]):
                cur_pred = y_pred[:,pollution]
                cur_obs = obs[:,pollution]
                loss = criterion(cur_pred[cur_obs!=-999],cur_obs[cur_obs!=-999])
                cur_loss.append(loss.cpu().data)
            ls.append(cur_loss)
    return np.mean(np.array(ls),axis = 0)

In [11]:

for epoch in range(25):
    start = time.time()
    print_log('-----------{}-----------'.format(epoch))
    ls = []
    
#     test_model.train()
    test_model.eval()
    test_model.rnn.train()
    for idx,i in enumerate(trainloader):
        
        indexes,input,grid,yt_1,label,obs = i[0],i[1],i[2],i[3],i[4],i[5]
        input,grid,yt_1,label,obs = input.to(device),grid.to(device),yt_1.to(device),label.to(device),obs.to(device)
        input = t2p(input)
        y_pred = test_model(input,grid,yt_1)

        if torch.sum(obs!=-999)==0:
            continue
        optimizer.zero_grad()
        
        loss = criterion(y_pred[obs!=-999],obs[obs!=-999])
        loss.backward()
        optimizer.step()
        ls.append(loss.cpu().data)
        
        #udpate input parameter
        Data.update(indexes,t2p.Input.data)
#         Data.update_labels(indexes,y_pred)
        if len(ls)%100==0:
            print_log('epoch {} cur loss {:.2f}'.format(epoch,np.mean(ls)))

    
    print_log('epoch {} is endding, the epoch training loss is : {:.2f}, time cost is {:.2f}'.format(epoch,np.mean(ls),time.time()-start))
    
    
    test_score = score(test_model,trainloader,criterion= nn.L1Loss())
    print_log('\n\n\n test score in the beginning is {}'.format(test_score))
    
    if epoch%3 == 0:
        for _idx,month in enumerate(['01'][:len(Data.EM)]):
            if not os.path.exists(EM_save_path):
                os.mkdir(EM_save_path)
            filename = EM_save_path+'/month_{}.npy'.format(month)
            np.save(filename,Data.EM[_idx])
            print_log(filename+'>>>>> input file save successful！')


-----------0-----------
epoch 0 cur loss 22.22
epoch 0 cur loss 22.10
epoch 0 cur loss 22.04
epoch 0 is endding, the epoch training loss is : 22.14, time cost is 574.85



 test score in the beginning is [41.23981  19.088596 13.095436 13.4019  ]
/scratch/AS_data/finetuneInput_Trust0.1/month_01.npy>>>>> input file save successful！
-----------1-----------
epoch 1 cur loss 22.20
epoch 1 cur loss 21.68
epoch 1 cur loss 21.62
epoch 1 is endding, the epoch training loss is : 21.51, time cost is 571.30



 test score in the beginning is [39.75508  18.780027 12.891012 13.267322]
-----------2-----------
epoch 2 cur loss 21.02
epoch 2 cur loss 21.19
epoch 2 cur loss 21.04
epoch 2 is endding, the epoch training loss is : 20.99, time cost is 572.47



 test score in the beginning is [38.78024  18.570787 12.732818 13.105587]
-----------3-----------
epoch 3 cur loss 20.90
epoch 3 cur loss 20.53
epoch 3 cur loss 20.60
epoch 3 is endding, the epoch training loss is : 20.49, time cost is 572.17



 tes

In [17]:
test_score = score(test_model,trainloader,criterion= nn.L1Loss())
print_log('\n\n\n test score in the beginning is {}'.format(test_score))





 test score in the beginning is [42.951664 19.542427 13.352887 13.638275]


In [10]:
?F.adam

Signature:
F.adam(
    params: List[torch.Tensor],
    grads: List[torch.Tensor],
    exp_avgs: List[torch.Tensor],
    exp_avg_sqs: List[torch.Tensor],
    max_exp_avg_sqs: List[torch.Tensor],
    state_steps: List[int],
    amsgrad: bool,
    beta1: float,
    beta2: float,
    lr: float,
    weight_decay: float,
    eps: float,
)
Docstring:
Functional API that performs Adam algorithm computation.

See :class:`~torch.optim.Adam` for details.
File:      ~/.local/lib/python3.7/site-packages/torch/optim/functional.py
Type:      function
